[🥭 Entropy Markets](https://entropy.markets/) support is available at: [Docs](https://docs.entropy.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/entropymarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=PlaceAndCancelOrder.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=PlaceAndCancelOrder.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Place And Cancel Order

This notebook shows how to place and cancel an order on a market.

This code will place an order on SOL-PERP, wait for it to confirm, and then cancel it.

One important thing to note is how the order is tracked. The `order` is built, passed to `place_order()` to place it, and then that same object is passed to `cancel_order()` to cancel it. The ID that ties the order built here and the order on-chain is the `client_id`.

A `client_id` is an integer you can specify when you create the `Order` object, and it stays with the order throughout its life. The `client_id` appears in orders on the `OrderBook`, and appears in events in the `EventQueue`. Unlike `id`, you choose what to use as the `client_id`, and you know what it is before the order is placed. (`id` is set by the system and you can't reliably know what it is before you send the order.)

If you don't specify a `client_id`, it is set to a timestamp.

In [ ]:
import decimal
import entropy

from solana.publickey import PublicKey

# Use our hard-coded mainnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file. Later we use
# its Entropy Account at AX7P7YXUNj7Ycp5VG5tweZqYngTk4kNFNZ7Zd2XHDrxB.
wallet = entropy.Wallet(bytes([181,213,227,47,41,229,109,138,15,82,26,7,230,184,88,102,197,215,238,155,136,196,138,92,98,154,67,68,47,140,90,40,248,149,223,193,241,51,4,196,126,32,211,66,90,137,249,160,132,246,38,29,88,16,252,116,12,83,117,158,40,98,178,54]))

with entropy.ContextBuilder.build(cluster_name="mainnet") as context:
    group = entropy.Group.load(context)
    account = entropy.Account.load(context, PublicKey("AX7P7YXUNj7Ycp5VG5tweZqYngTk4kNFNZ7Zd2XHDrxB"), group)
    market_operations = entropy.operations(context, wallet, account, "SOL-PERP", dry_run=False)

    print("Orders (initial):")
    print(market_operations.load_orderbook())

    order = entropy.Order.from_values(side=entropy.Side.BUY,
                                    price=decimal.Decimal(10),
                                    quantity=decimal.Decimal(1),
                                    order_type=entropy.OrderType.POST_ONLY,
                                    client_id=1001)
    print("Placing order:", order)
    placed_order_signatures = market_operations.place_order(order)

    print("Waiting for place order transaction to confirm...\n", placed_order_signatures)
    entropy.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, placed_order_signatures, commitment="processed"
        )

    print("\n\nOrders (including our new order):")
    orderbook = market_operations.load_orderbook()
    print(orderbook)

    # Order has the client ID 1001 so we can use that Order object as a parameter here.
    cancellaton_signatures = market_operations.cancel_order(order)

    print("Waiting for cancel order transaction to confirm...\n", cancellaton_signatures)
    entropy.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, cancellaton_signatures, commitment="processed"
        )

    print("\n\nOrders (without our order):")
    print(market_operations.load_orderbook())

print("Example complete.")